## Python 에서 GC 사용법

### gc 모듈 불러오기

In [2]:
import gc

### get_threshold() 메소드
GC에 구성된 세대(generation)별 임계값(threshold) 확인

In [3]:
gc.get_threshold()

(700, 10, 10)

### get_count() 메소드
0 ~ 2 세대에 속한 객체 수 확인

In [4]:
gc.get_count()

(444, 0, 11)

### collect() 메소드
수동으로 GC 프로세스를 실행하여 세대별 객체 정리

In [5]:
gc.collect()

254

In [6]:
gc.get_count()

(207, 0, 0)

### set_threshold() 메소드
GC에 구성된 세대(generation)별 임계값(threshold) 조정

In [10]:
gc.set_threshold(1000, 15, 15)

In [11]:
gc.get_threshold() # (700, 10, 10) -> (1000, 15, 15)

(1000, 15, 15)

## GC 사용시에도 메모리 누수가 발생하는 경우

### 1. 순환참조
특정 객체가 서로를 참조(순환참조)하고 있다면 Reference counting 방식으로는 객체를 메모리에서 해제할 수 없다.

In [21]:
import sys

temp = []
temp.append(temp)
print(sys.getrefcount(temp))

del temp
print(sys.getrefcount(temp))

3


NameError: name 'temp' is not defined

### 2. 미사용 객체에 대한 트래킹 미작동으로 메모리가 해제되지 않은 경우
MyClass 클래스에서 객체를 생성할 때마다, 새롭게 메모리에 할당되며, 해당 객체의 참조를 리스트가 가지고 있으므로, 

리스트가 메인 스코프에서 삭제되기 전까지는 GC의 대상이 될 수 없으며, 메모리 누수가 발생할 수 있습니다.

만약, 루프 반복이 많아질 경우 메모리 사용량이 크게 증가하여 다른 프로세스에 영향을 줄 수 있습니다.

In [36]:
class MyClass:
   def __init__(self):
       self.x = 42

def create_instance():
    instance = MyClass()
    return instance

def main():
    instances = []
    for _ in range(10):
        instances.append(create_instance())
    return instances

main()

### 객체를 사용한 이후에 메모리에서 직접 삭제하여 해결
사용 후 필요 없어진 객체를 리스트 내에서 수동으로 삭제하여 불필요하게 메모리를 점유하는 객체를 해제했습니다.

In [40]:
class MyClass:
   def __init__(self):
       self.x = 42

def main():
    instances = []
    for _ in range(10):
        instances.append(create_instance())

    # using...

    for instance in instances:
        del instance

    del instances

main()

### 제너레이터 함수 사용으로 객체의 생성을 제어하여 해결
get_instance() 함수는 제너레이터(generator) 함수이므로, yield 로 반환한 객체들은 계속해서 생성 후에 사라지게 됩니다.

제너레이터 객체를 통해 생성된 객체는 다음 값 생성 시점에 대해 더 이상 유효하지 않기 때문에 즉각적으로 GC 대상이 되며, 따라서 메모리 누수가 발생하지 않습니다.

In [38]:
class MyClass:
    def __init__(self):
        self.x = 42

def get_instance():
    while True:
        yield MyClass()

def main():
    instances = []
    generator = get_instance()
    for _ in range(10):
        instances.append(next(generator))
    return instances

main()